## 第9章 魔法方法， 属性， 迭代器
* 构造(初始化) \__init\__
* 成员访问
* 属性
* 迭代器
* 生成器
* 案例（八皇后问题）

### 构造 \__init__
* 基本例子
* 包含继承关系的例子

In [1]:
# init 基本例子
class fooBar(object):
    def __init__(self, value = 0):
        self.somevar = value

if __name__=="__main__":
    f1 = fooBar()
    print(f1.somevar)
    f2 = fooBar(45)
    print(f2.somevar)
    
# init 包含多类问题的例子
# 父类
class Bird(object):
    def __init__(self):
        self.hungry = True
    def eat(self):
        if self.hungry:
            print("Aaahhh")
            self.hungry = False
        else:
            print("No, I m full")
# 子类
class SingBird(Bird):
    def __init__(self):
        super(SingBird, self).__init__()
        self.sound = "I love u"
    def sing(self):
        print(self.sound)

if __name__=="__main__":
    bird = Bird()
    bird.eat()
    bird.eat()
    
    singbird = SingBird()
    singbird.eat()
    singbird.eat()
    singbird.sing()

0
45
Aaahhh
No, I m full
Aaahhh
No, I m full
I love u


### 成员访问（主要是list）
* \__len\__(self): len(list)
* \__getitem\__(self): a[2]
* \__setitem\__(self): a[2]=9
* \__delitem\__(self): del a[2]

In [2]:
def checkIndex(key):
    """
    The key must be int and >0
    """
    if not isinstance(key, int):    # 在实际中尽量不使用isinstance， 因为和多态相悖
        raise TypeError
    if key < 0: 
        raise IndexError
        
class ArithmSeq(object):
    def __init__(self, start = 0, step = 1):
        self.start = start
        self.step = step
        self.seq = {}
    def __getitem__(self, key):
        checkIndex(key)
        try:
            return self.seq[key]
        except KeyError:
            return self.start + key * self.step
    def __setitem__(self, key, value):
        checkIndex(key)
        self.seq[key] = value 
        
if __name__=="__main__":
    s = ArithmSeq(1,2)
    print(s[4])
    s[2] = 8
    print(s[2])   

9
8


* 自定义自己的list, 采用继承的方式。因为实现多态性是件困难的事情，因此可以利用python自带的list特性

In [3]:
class CountList(list):
    def __init__(self,*args):
        super(CountList, self).__init__(*args)
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        return super(CountList, self).__getitem__(index)

if __name__ == "__main__":
    clist = CountList(range(10))
    print(clist)
    print(clist.counter)
    print(clist[2])
    print(clist.counter)
    print(clist[5])
    print(clist.counter)
    del clist[2:5]
    print(clist)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
2
1
5
2
[0, 1, 5, 6, 7, 8, 9]


### 属性(property)
* 简单来说，就是把函数作为属性访问，即可以直接用. 访问的, 不关心内部是怎么实现的

In [4]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def setSize(self, size):
        self.width, self.height = size
    def getSize(self):
        return self.width, self.height
    def getArea(self):
        return self.width*self.height
    size = property(getSize, setSize)
    area = property(getArea)

if __name__=="__main__":
    r = Rectangle()
    r.width = 12
    r.height = 30
    print(r.area)
    print(r.size)

360
(12, 30)


### 迭代器

In [5]:
# 运行时先调用__iter__, 再调用next
class Fibs:
    def __init__(self):
        self.a, self.b = 0, 1
    def __next__(self):
        self.a, self.b  = self.b, self.a + self.b
        if self.a > 10: 
            raise StopIteration
        return self.a
    def __iter__(self):
        return self
    
if __name__=="__main__":
    fibs = Fibs()
    print(list(fibs))
    for i in fibs:  # 无输出的原意是已经rasie StopIteration了
        if(i<20):
            print(i)
        else:
            break
    

[1, 1, 2, 3, 5, 8]


### 生成器
* 任务1： 解析一层列表，且只有数字[[1,2],[3,4],[5]] -> [1,2,3,4,5] 
* 任务2： 解析多层列表，且只有数字[[[1],2],3,4,[5,[6,7],8]] -> [1,2,3,4,5,6,7,8]
* 任务3： 解析包含字符串列表 ['foo',['bar',['baz']]] -> ['foo', 'bar', 'baz']
* Tip: send 的使用

In [6]:
def flatten1(nested):
    for layer1 in nested:
        for layer2 in layer1:
            yield layer2
def flatten2(nested):
    try:
        for layer1 in nested:
            for layer2 in flatten2(layer1):
                yield layer2                #必须要加yield, 返回值是一个flatten过的生成器，返回值不返回到最外层的函数
    except TypeError:
        yield nested

def flatten3(nested):
    try:
        try: 
            nested + ''
            # raise TypeError              # 放在这里是错误的，会引起死循环
        except TypeError: 
            pass
        else:
            raise TypeError
        for layer1 in nested:
            for layer2 in flatten3(layer1):
                yield layer2
    except TypeError:
        yield nested

def repeater(value):
    while True:
        new = yield value
        if new is not None:
            value = new

if __name__=="__main__":
    nest1 = [[1,2],[3,4],[5]]
    print(list(flatten1(nest1)))

    nest2 = [[[1],2],3,4,[5,[6,7],8]]
    print(list(flatten2(nest2)))

    nest3 = ['foo',['bar',['baz']]]
    print(list(flatten3(nest3)))
    
    # send 直接传值给value, 然后执行yield下面的语句，直至碰到下一个yield，产生下一个值。
    r = repeater(42)  # r 是生成器
    print(next(r))    # 返回数值
    r.send("hello")
    print(next(r))

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 6, 7, 8]
['foo', 'bar', 'baz']
42
hello


### 迭代器与生成器的一些思考 
* 迭代器需建类，生成器自动包含迭代，不用创建类


### 案例（八皇后问题）
#### 问题描述：n 皇后， 则NxN方格，n个皇后不能在同一行，同一列或者对角线上，请问有几种放法。
#### 解题思路（以四皇后为例）
* 状态表示: 如果可以成功放置的话，肯定每一行都有一个皇后，因此可以按顺序放置，并且只记录皇后的放置的列。 例如：state[0]=1，第一个皇后在第一行第一列。
* 寻找冲突：如果有冲突，返回True, 否则返回False.
* 基本情况：假设前面三个皇后都已经放好了
* 需要递归的情况： 时刻记住函数返回的是什么，如果达到state 达到 num_queen，则依次从for返回上一层

In [7]:
# 定义冲突
def conflict(state,pos):
    num_already = len(state)
    for i in range(num_already):
        if(pos == state[i]): 
            return True
        if(abs(i - num_already) == abs(state[i] - pos)):
            return True
    return False

# 基本情况
def queen(num_queen, state=[1,3,0]):
    if len(state) == num_queen-1:
        for pos in range(num_queen):
            if not conflict(state, pos):
                yield pos

# 需要递归的情况 --- 时刻记住返回的是什么。
def queenD(num_queen, state):
    for pos in range(num_queen):
        if not conflict(state,pos):
            if len(state) == num_queen-1:
                yield (pos,)
            else:
                for result in queenD(num_queen, state+[pos,]):
                    yield (pos,) + result

if __name__=="__main__":
    print(list(queen(4)))
    print(list(queenD(4,[])))


[2]
[(1, 3, 0, 2), (2, 0, 3, 1)]


In [8]:
list(queenD(4,[]))

[(1, 3, 0, 2), (2, 0, 3, 1)]

In [9]:
(2,)+(4,)

(2, 4)